In [15]:
import os 
import pickle

print(os.listdir(os.path.join(os.getcwd(),"deterministic_long_policy", "deterministic_long_policy_g=8_a=8_t=25")))

p = os.path.join(os.getcwd(),"deterministic_long_policy",
                            "deterministic_long_policy_g=8_a=8_t=25","deterministic_long_policy_g=8_a=8_t=25.pkl")

pt = r"C:\\Users\\james\\MSCIActiveInferenceProject\\results\\deterministic_long_policy\\deterministic_long_policy_g=8_a=8_t=25\\deterministic_long_policy_g=8_a=8_t=25.pkl"
with open(p, "rb") as file:
    pkl_file = pickle.load(file)

['deterministic_long_policy_g=8_a=8_t=25.pkl', 'deterministic_long_policy_g=8_a=8_t=25.yml']


In [34]:
import os
import yaml
import pathlib

def tuple_constructor(loader, node):
    return tuple(loader.construct_sequence(node))

yaml.add_constructor('tag:yaml.org,2002:python/tuple', tuple_constructor)

def yml_loader(path, exp_name, show):
    with open(path, "r") as file:
        yaml_file = yaml.load(file, Loader=yaml.FullLoader)
    indep = yaml_file["parameters"]
    
    # indep_output = f"a={indep['alpha']}_g={indep['gamma']}_t={indep['time_steps']}"
    # dep_output = f"peak_memory={float(yaml_file['peak_memory_per_episode']):.2f}\ntime_per_episode={float(yaml_file['time_per_episode']):.2f}\npolicy_length_per_episode={float(yaml_file['policy_length_per_episode']):.2f}\nstate_space_coverage={float(yaml_file['state_space_coverage']):.2f}\nepisodes={yaml_file['episode_count']}"
    
    ### independent results
    alpha = indep['alpha']
    gamma = indep['gamma']
    time_steps = indep['time_steps']
    
    ### Dependent Results
    
    peak_memory = f"{float(yaml_file['peak_memory_per_episode']/1024):.2f}" # megabytes
    time_per_episode = f"{float(yaml_file['time_per_episode']):.2f}"
    pol_len_per_episode = f"{float(yaml_file['policy_length_per_episode']):.2f}"
    state_space_coverage = f"{float(yaml_file['state_space_coverage']):.2f}"
    episode_count = f"{yaml_file['episode_count']}"
    upper_bound_memory = f"{(float(episode_count) * float(peak_memory)):.2f}"
    total_elapsed_time = f"{(float(episode_count) * (float(time_per_episode)/60)):.2f}" 
    
    indep_latex_row = f"{alpha} {gamma} {time_steps}"
    dep_latex_row = f"{peak_memory} & {time_per_episode} & {pol_len_per_episode} & {state_space_coverage} & {episode_count} & {upper_bound_memory} & {total_elapsed_time}"
    
    latex_row = f"{indep_latex_row} & & {dep_latex_row} \\\\"
    alpha_gamma_show = [(8, 8), (16, 16), (32, 32)]
    time_show = [150, 175]
    
    if show:
        if (alpha, gamma) in alpha_gamma_show and time_steps in time_show:
            print(exp_name)
            print(latex_row)
    else:
        print(latex_row)
    

In [35]:
# derive yaml data from results
import os
import yaml
import pathlib
import re

aif_results = os.path.join(os.getcwd(),"AiF_real")

def extract_sort_key(filename):
    match = re.search(r'g=(\d+)_a=(\d+)_t=(\d+)', filename)  # Extract numbers
    if match:
        g, a, t = map(int, match.groups())  # Convert to integers
        return (g, a, t)  # Sort by g, then a, then t
    return (0, 0, 0)  # Default if no match found

def extract_experiments_clean(folder, exp, show):
    """
    Given an experiment name, including the folder where it is, extract the results and present them
    :return: 
    """
    exp_folder = os.path.join(os.getcwd(), folder, exp)
    
    for exp_class in sorted(os.listdir(exp_folder)):
        exp_class_path = os.path.join(exp_folder,exp_class)
        for dec_model in sorted(os.listdir(exp_class_path), key=extract_sort_key ):
            dec_model_path = os.path.join(exp_class_path,dec_model, dec_model + ".yml")
            yml_loader(dec_model_path,  exp_class_path + " " + dec_model, show)
    

# for exp_dir in os.listdir(aif_results):
#     exp_dir_path = os.path.join(aif_results,exp_dir)
#     for exp_class in os.listdir(exp_dir_path):
#         exp_class_path = os.path.join(exp_dir_path,exp_class)
#         for dec_model in os.listdir(exp_class_path):
#             dec_model_path = os.path.join(exp_class_path,dec_model, dec_model + ".yml")
#             yml_loader(dec_model_path,  exp_dir + " " + dec_model)
                
        



In [36]:
folder = os.path.join("AiF_real", "performance")

In [37]:
"""
5x5 experiment performance results
"""
exp_5x5 = "5_by_5_grid_experiment"

extract_experiments_clean(folder, exp_5x5, True)

C:\Users\james\MSCIActiveInferenceProject\results\AiF_real\performance\5_by_5_grid_experiment\deterministic_long_policy deterministic_long_policy_g=8_a=8_t=150
8 8 150 & & 4.19 & 17.78 & 8.22 & 0.27 & 18 & 75.42 & 5.33 \\
C:\Users\james\MSCIActiveInferenceProject\results\AiF_real\performance\5_by_5_grid_experiment\deterministic_long_policy deterministic_long_policy_g=16_a=16_t=150
16 16 150 & & 4.36 & 21.26 & 10.00 & 0.33 & 14 & 61.04 & 4.96 \\
C:\Users\james\MSCIActiveInferenceProject\results\AiF_real\performance\5_by_5_grid_experiment\deterministic_long_policy deterministic_long_policy_g=32_a=32_t=150
32 32 150 & & 4.30 & 20.86 & 9.73 & 0.31 & 15 & 64.50 & 5.21 \\
C:\Users\james\MSCIActiveInferenceProject\results\AiF_real\performance\5_by_5_grid_experiment\stochastic_long_policy stochastic_long_policy_g=8_a=8_t=150
8 8 150 & & 4.31 & 26.96 & 12.50 & 0.36 & 12 & 51.72 & 5.39 \\
C:\Users\james\MSCIActiveInferenceProject\results\AiF_real\performance\5_by_5_grid_experiment\stochastic_lon

In [29]:
"""
7x7 experiment performance results
"""
exp_7x7 = "7_by_7_grid_experiment"



In [24]:
"""
9x9 experiment performance results
"""
exp_9x9 = "9_by_9_grid_experiment"

extract_experiments_clean(folder, exp_9x9)

C:\Users\james\MSCIActiveInferenceProject\results\AiF_real\performance\9_by_9_grid_experiment\stochastic_long_long_policy stochastic_long_long_policy_g=16_a=16_t=125
a=16_g=16_t=125
peak_memory=26596.54
time_per_episode=2580.00
timesteps_episode=36.00
policy_length_per_episode=36.00
state_space_coverage=0.33
episodes=2


C:\Users\james\MSCIActiveInferenceProject\results\AiF_real\performance\9_by_9_grid_experiment\stochastic_long_long_policy stochastic_long_long_policy_g=16_a=16_t=175
a=16_g=16_t=175
peak_memory=53637.34
time_per_episode=2607.58
timesteps_episode=57.00
policy_length_per_episode=57.00
state_space_coverage=0.46
episodes=2


C:\Users\james\MSCIActiveInferenceProject\results\AiF_real\performance\9_by_9_grid_experiment\stochastic_long_long_policy stochastic_long_long_policy_g=16_a=16_t=250
a=16_g=16_t=250
peak_memory=82748.10
time_per_episode=5020.87
timesteps_episode=108.00
policy_length_per_episode=108.00
state_space_coverage=0.54
episodes=2


C:\Users\james\MSCIActiveInfe

KeyboardInterrupt: 

In [26]:
"""
spiral maze experiment performance results
"""

maze_path_9x9 =  "maze_9_by_9"
extract_experiments_clean(folder, maze_path_9x9)


TypeError: float() argument must be a string or a number, not 'list'